# Quick Start

We now showcase some basic functionality present in CoLA. We'll start by showing how to define different types of Linear ops, then we'll show how to perform basic arithmetic with Linear ops and, finally, we'll conclude applying some linear algebra operations (like solves or log determinants) to the Linear ops.

We'll work with torch.tensors in this example, but the same code can be run using JAX arrays (jnp.ndarrays). 

In [1]:
import cola
import torch
import numpy as np
import warnings

warnings.filterwarnings("ignore")
torch.manual_seed(21)

## Creating a Linear Operator

You can find several predefined Linear ops under cola.ops. We'll ilustrate three basic cases: Dense, Diagonal and Tridiagonal.

A Dense Linear Operator is nothing more than a wrapper on a dense matrix, where the wrapper defines a matmat function $v \mapsto Av$ and holds several attributes such as dtype and shape.

Let's start by defining a dense matrix and a vector to act upon. Below we show the entries of the matrix $A$, of the vector $v$ and the result of $Av$.

In [2]:
N = 4
A_dense = torch.randn(N, N)
vec = torch.randn(N)
print(f"A: {A_dense}\nAv: {A_dense @ vec}")

A: tensor([[-0.2386, -1.0934,  0.1558,  0.1750],
        [-0.9526, -0.5442,  1.1985,  0.9604],
        [-1.1074, -0.8403, -0.0020,  0.2240],
        [ 0.8766, -0.5379, -0.2994,  0.9785]])
Av: tensor([-0.3842, -1.8328, -0.7573,  0.9795])


To create a Dense operator simply run:

In [3]:
A = cola.ops.Dense(A_dense)
print(type(A))

<class 'cola.ops.operators.Dense'>


The previous operator now has a dtype and a shape attribute. More importantly, it can now act on the vector $v$ and get the same result as above.

In [4]:
print(f"Dtype: {A.dtype} | Shape: {A.shape}")
print(A @ vec)

Dtype: torch.float32 | Shape: torch.Size([4, 4])
tensor([-0.3842, -1.8328, -0.7573,  0.9795])


In order to make use of the benefits of CoLA, let's consider using some large structured Linear Operators. First let's start by creating some structured linear operators, such as a digaonal, a dense matrix, and a permutation matrix.

In [5]:
ones = torch.ones(300)
upper = ones[:-1]
lower = ones[:-1]
diagonal= -2*ones

T = cola.ops.Tridiagonal(lower, diagonal, upper)
print(f"{T[:3,:3].to_dense()}, Type: {T}")

tensor([[-2.,  1.,  0.],
        [ 1., -2.,  1.],
        [ 0.,  1., -2.]]), Type: Tridiagonal


In [6]:
perm = torch.randperm(200)
P = cola.ops.Permutation(perm)
v = torch.randn(200)
print((P @ v)[:5], v[perm[:5]])

tensor([-0.4161, -0.2753,  1.9718,  0.0790, -0.1104]) tensor([-0.4161, -0.2753,  1.9718,  0.0790, -0.1104])


For these Linear Operators, the memory and compute cost for multiplication is only $O(n)$ rather than the $O(n^2)$ for dense.

## Doing binary operations with Linear ops

CoLA provides a similar interface to combine Linear ops as you would combine matrices. 
We can add and subtract (+,-), multiply by scalars (*,/), matrix multiply (@), transpose (.T), slice [:,:] and more

In [7]:
D = cola.diag(torch.tensor([1, 2, 3, 4.]))
DT = T[:4, :4] - D
print(DT.to_dense())

tensor([[-3.,  1.,  0.,  0.],
        [ 1., -4.,  1.,  0.],
        [ 0.,  1., -5.,  1.],
        [ 0.,  0.,  1., -6.]])


Let's use these operations to make $B= A^T(T_{[:4,:4]}-D) + \mu I$ regularized by $\mu$ by running the following code.

In [8]:
mu = 1e-6
B = A.T @ DT
B += mu * cola.ops.I_like(B)
print(f"Object: {B}\n {B.to_dense()}")

Object: DenseTridiagonal[slice(None, 4, None),slice(None, 4, None)]+-1.0diag(tensor([1., 2., 3., 4.]))+9.999999974752427e-07I
 tensor([[-0.2368,  2.4642,  5.4611, -6.3671],
        [ 2.7361,  0.2431,  3.1194,  2.3872],
        [ 0.7310, -4.6402,  0.9091,  1.7946],
        [ 0.4354, -3.4425,  0.8188, -5.6472]])


Linear Operators can also be combined in more involved ways, such as with `kron` for $\otimes$, `block_diag`, and `concatenate`

In [9]:
K = cola.kron(A, D)
print(f"{K[:5,:5].to_dense()}\n shape: {K.shape}, object: {K}")

tensor([[-0.2386,  0.0000,  0.0000,  0.0000, -1.0934],
        [ 0.0000, -0.4772,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.7157,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.9543,  0.0000],
        [-0.9526,  0.0000,  0.0000,  0.0000, -0.5442]])
 shape: (16, 16), object: Dense⊗diag(tensor([1., 2., 3., 4.]))


In [10]:
d = 1 / (1 + torch.linspace(.2, 10, 1000))
X = cola.block_diag(cola.kron(A, P), T, cola.diag(d))
X

<2100x2100 BlockDiag[cola.ops.operators.Kronecker[cola.ops.operators.Dense, cola.ops.operators.Permutation], cola.ops.operators.Tridiagonal, cola.ops.operators.Diagonal] with dtype=torch.float32>

Under the hood the operator $B$ is lazily defined and would know how to apply the Linear ops to any vector $v$.

## Computing linear algebra operations (e.g. solves, logdet, diag, exp)

To solve the linear system $Bx=v$ we use the inverse function. This inverse function lazily defines $B^{-1}$ and hence applying it to $v$ yields the solution $x=B^{-1}v$. The inverse of $B$ is never computed, using $B^{-1}$ is simply how in CoLA we call linear solves.

In [11]:
X[:, :5].to_dense().shape

torch.Size([2100, 5])

In [12]:
X_inv = cola.inv(X)
v = torch.randn(X.shape[-1])
soln = X_inv @ v
print(f"Solution error: {torch.linalg.norm(X @ soln - v)/torch.linalg.norm(v):.2e}")

Solution error: 4.37e-06


In [13]:
print(f"CoLA logdet: {cola.logdet(X):.3f}, dense logdet {torch.logdet(X.to_dense()):.3f}")

CoLA logdet: -1564.690, dense logdet -1564.690


In [14]:
print(f"CoLA diag: {cola.diag(X)[-5:]}\ndense diag: {torch.diag(X.to_dense())[-5:]}")

CoLA diag: tensor([0.0912, 0.0912, 0.0911, 0.0910, 0.0909])
dense diag: tensor([0.0912, 0.0912, 0.0911, 0.0910, 0.0909])


In [15]:
from scipy.linalg import expm
scipy_expm = expm((T.to_dense()).numpy())
print(f"CoLA exp(X): {cola.exp(T)[-5:,-5:].to_dense().real}\nscipy expm: {scipy_expm[-5:,-5:]}")

CoLA exp(X): tensor([[0.3085, 0.2153, 0.0932, 0.0288, 0.0066],
        [0.2153, 0.3085, 0.2152, 0.0930, 0.0275],
        [0.0932, 0.2152, 0.3083, 0.2139, 0.0864],
        [0.0288, 0.0930, 0.2139, 0.3016, 0.1865],
        [0.0066, 0.0275, 0.0864, 0.1865, 0.2153]])
scipy expm: [[0.30850834 0.21526882 0.09323522 0.02876082 0.0066488 ]
 [0.21526891 0.30850452 0.21523876 0.09302247 0.02746145]
 [0.09323531 0.21523881 0.30829167 0.21393949 0.08637363]
 [0.02876082 0.09302244 0.21393946 0.30164295 0.18647799]
 [0.0066488  0.02746145 0.08637364 0.18647805 0.21526927]]


In [16]:
print(f"CoLA exp(X): {cola.exp(T, method='iterative')[-5:,-5:].to_dense().real}\nscipy expm: {scipy_expm[-5:,-5:]}")

CoLA exp(X): tensor([[0.3085, 0.2153, 0.0932, 0.0288, 0.0066],
        [0.2153, 0.3085, 0.2152, 0.0930, 0.0275],
        [0.0932, 0.2152, 0.3083, 0.2139, 0.0864],
        [0.0288, 0.0930, 0.2139, 0.3016, 0.1865],
        [0.0066, 0.0275, 0.0864, 0.1865, 0.2153]])
scipy expm: [[0.30850834 0.21526882 0.09323522 0.02876082 0.0066488 ]
 [0.21526891 0.30850452 0.21523876 0.09302247 0.02746145]
 [0.09323531 0.21523881 0.30829167 0.21393949 0.08637363]
 [0.02876082 0.09302244 0.21393946 0.30164295 0.18647799]
 [0.0066488  0.02746145 0.08637364 0.18647805 0.21526927]]


In [17]:
C = X.T @ X
# let's annotate this matrix as positive definite to speed up the computation
C = cola.PSD(C)
e, v = cola.eig(C, eig_slice=np.s_[-3:], max_iters=100)

In [18]:
print(e, torch.linalg.eigh(C.to_dense())[0][-3:])

tensor([1.0953e-03, 1.0964e-02, 2.1969e-02, 4.2399e-02, 7.0133e-02, 1.0801e-01,
        1.4944e-01, 1.9419e-01, 2.5152e-01, 3.1772e-01, 4.0139e-01, 4.7665e-01,
        5.6189e-01, 6.4727e-01, 7.3868e-01, 8.8262e-01, 9.8502e-01, 1.1289e+00,
        1.2921e+00, 1.4241e+00, 1.5924e+00, 1.7201e+00, 1.9020e+00, 2.0731e+00,
        2.2735e+00, 2.3025e+00, 2.5105e+00, 2.8178e+00, 2.9081e+00, 3.2181e+00,
        3.3807e+00, 3.6097e+00, 3.7622e+00, 3.9949e+00, 4.2446e+00, 4.4903e+00,
        4.6545e+00, 4.9160e+00, 5.1424e+00, 5.3507e+00, 5.5901e+00, 5.7817e+00,
        6.0946e+00, 6.3530e+00, 6.6405e+00, 6.8644e+00, 7.1608e+00, 7.3553e+00,
        7.5980e+00, 7.7476e+00, 8.1347e+00, 8.3219e+00, 8.5606e+00, 8.8394e+00,
        9.1587e+00, 9.3410e+00, 9.6295e+00, 9.8448e+00, 1.0079e+01, 1.0330e+01,
        1.0445e+01, 1.0769e+01, 1.1044e+01, 1.1270e+01, 1.1530e+01, 1.1755e+01,
        1.1908e+01, 1.2241e+01, 1.2330e+01, 1.2568e+01, 1.2806e+01, 1.2999e+01,
        1.3167e+01, 1.3295e+01, 1.3514e+

In [19]:
e, v = cola.eig(C, eig_slice=np.s_[:3])
print(e, torch.linalg.eigh(C.to_dense())[0][:3])

tensor([-2.0163e-07,  7.4170e-07,  2.9827e-06,  6.1287e-06,  8.6353e-06,
         1.5342e-05,  2.6599e-05,  4.8417e-05,  7.6746e-05,  1.1866e-04,
         1.7418e-04,  2.4626e-04,  3.3815e-04,  4.5447e-04,  5.9902e-04,
         7.7438e-04,  9.8632e-04,  1.2406e-03,  1.5368e-03,  1.8854e-03,
         2.2899e-03,  2.7560e-03,  3.2899e-03,  3.8961e-03,  4.5839e-03,
         5.3573e-03,  6.2235e-03,  7.1915e-03,  8.2694e-03,  8.2759e-03,
         8.3232e-03,  8.3387e-03,  8.3862e-03,  8.4328e-03,  8.4770e-03,
         8.5494e-03,  8.6306e-03,  8.6876e-03,  8.8099e-03,  8.8861e-03,
         9.0070e-03,  9.1049e-03,  9.2122e-03,  9.3541e-03,  9.4663e-03,
         9.6082e-03,  9.7719e-03,  9.8973e-03,  1.0062e-02,  1.0219e-02,
         1.0413e-02,  1.0577e-02,  1.0742e-02,  1.0947e-02,  1.1148e-02,
         1.1309e-02,  1.1519e-02,  1.1710e-02,  1.1989e-02,  1.2169e-02,
         1.2379e-02,  1.2585e-02,  1.2774e-02,  1.3063e-02,  1.3274e-02,
         1.3599e-02,  1.3839e-02,  1.4106e-02,  1.4